# Huấn luyện Transformer trên Kaggle

Notebook này được thiết kế để chạy trên Kaggle.

## 1. Thiết lập Môi trường
Kaggle có thư mục `input` là chỉ đọc (read-only). Để train model, lưu log và checkpoint, ta cần copy code sang thư mục `working`.

In [ ]:
import os
import shutil

# Giả sử bạn upload Dataset có tên 'transformer' chứa toàn bộ code
# Đường dẫn thường là /kaggle/input/transformer
INPUT_PATH = '/kaggle/input/transformer'
WORKING_PATH = '/kaggle/working/Transformer'

if os.path.exists(INPUT_PATH):
    if os.path.exists(WORKING_PATH):
        shutil.rmtree(WORKING_PATH)
    
    print("Đang copy code sang /kaggle/working để có quyền ghi...")
    shutil.copytree(INPUT_PATH, WORKING_PATH)
    os.chdir(WORKING_PATH)
    print(f"Đã chuyển vào: {os.getcwd()}")
else:
    print(f"Không tìm thấy {INPUT_PATH}. Hãy chắc chắn bạn đã Add Data là folder code của mình.")
    # Nếu bạn upload file thủ công chứ không phải dataset, có thể bạn đang ở sẵn /kaggle/working
    print("Kiểm tra thư mục hiện tại:")
    print(os.listdir('.'))

## 2. Cài đặt Thư viện
Cài các thư viện cần thiết.

In [ ]:
!pip install -r requirements.txt

## 3. Xử lý Dữ liệu Clean (Tùy chọn)
Nếu bạn chỉ upload dữ liệu Raw và muốn chạy clean data trên Kaggle, hãy chạy cell này.
Nó sẽ đọc từ `data/raw` và tạo ra `data/clean`.

In [ ]:
# Chạy script clean data
!python scripts/clean_data.py

## 4. Tùy chỉnh Cấu hình
Bạn có thể thay đổi tham số training tại đây.

**Lưu ý về Inference (Decoding):**
- **Greedy Search (`beam_size: 1`, `decoding_method: "greedy"`)**: Nhanh, nhẹ, chọn từ có xác suất cao nhất tại mỗi bước. Phù hợp debug.
- **Beam Search (`beam_size: 5`, `decoding_method: "beam"`)**: Chậm hơn nhưng chất lượng dịch tốt hơn, biết chọn phương án tối ưu toàn cục. Dùng khi Test lấy điểm BLEU.
  - Chú ý: Phải sửa cả `beam_size > 1` VÀ `decoding_method: "beam"` thì Beam Search mới được kích hoạt.

In [ ]:
%%writefile configs/default.yaml
model:
  d_model: 512
  n_layers: 6
  heads: 8
  dropout: 0.1
  vocab_size: 32768 
  max_len: 128 # Tăng lên tùy bài toán

training:
  batch_size: 64 # Kaggle GPU P100 khá mạnh, có thể để 64
  lr: 0.0005 # An toàn
  epochs: 20
  warmup_ratio: 0.07 # 7% steps
  save_dir: "checkpoints"
  log_dir: "logs"
  seed: 42
  num_workers: 4 # Tăng lên 4 hoặc 8 trên Colab để load dữ liệu nhanh hơn

inference:
  # Cấu hình Decoding (Dịch)
  beam_size: 1             # 1 = Greedy (Mặc định). Tăng lên 5 để dùng Beam Search.
  decoding_method: "greedy" # "greedy" hoặc "beam".
  length_penalty: 0.6      # Phạt câu ngắn/dài khi dùng Beam Search.
  eval_batch_size: 64

data:
  min_len: 3
  max_ratio: 2.0
  bpe_dropout: 0.1

wandb:
  enabled: false
  project: "transformer-from-scratch"

## 5. Build Tokenizer (Bắt buộc)
Huấn luyện lại Tokenizer để đảm bảo tính tương thích (do chúng ta đã chuyển sang dùng thư viện `tokenizers` chuẩn).

In [ ]:
!python scripts/train_tokenizer.py --data_dir data/clean

## 6. Data Filtering & Caching (Optional)
Chạy script lọc dữ liệu để làm sạch, lọc (theo độ dài/tỷ lệ) và lưu đệm (cache) dữ liệu trước khi huấn luyện. Bước này là tùy chọn nhưng được khuyến nghị để đảm bảo chất lượng dữ liệu.

In [ ]:
# Run data filtering script
!python scripts/filter_data.py --force --export-text

## 7. Kiểm tra GPU
Kaggle thường cung cấp GPU P100 hoặc T4 x 2.

In [ ]:
!nvidia-smi

## 8. Huấn luyện (Training)
Bắt đầu quá trình huấn luyện.

In [ ]:
# import wandb
# wandb.login(key="PASTE_YOUR_API_KEY_HERE")

!python scripts/train.py

## 9. Kiểm tra Model (Test)
Chạy thử model trên tập Validation và Test với checkpoint vừa train.

In [ ]:
# Thay tên checkpoint bạn muốn test
checkpoint_path = "checkpoints/checkpoint_epoch_20.pt"
if os.path.exists(checkpoint_path):
    !python scripts/test_model.py --checkpoint {checkpoint_path}
else:
    print(f"Chưa thấy file {checkpoint_path}. Hãy kiểm tra lại folder checkpoints.")

## 10. Đánh giá BLEU Score (Test Only)
Tính điểm BLEU chuẩn quốc tế (SacreBLEU) trên tập Test, sử dụng Beam Search (Beam Size = 5) để có kết quả tốt nhất.

In [ ]:
checkpoint_path = "checkpoints/checkpoint_epoch_20.pt"
if os.path.exists(checkpoint_path):
    if os.path.exists("data/clean/test.en"):
        print("\n--- Đánh giá trên tập Test (Greedy) ---")
        !python scripts/calculate_bleu.py \
            --checkpoint {checkpoint_path} \
            --split test \
            --decoding_method greedy \
            --beam_size 1

    if os.path.exists("data/clean/test.en"):
        print("\n--- Đánh giá trên tập Test (Beam Search Size 5) ---")
        !python scripts/calculate_bleu.py \
            --checkpoint {checkpoint_path} \
            --split test \
            --decoding_method beam \
            --beam_size 5
    else:
        print("Không tìm thấy tập test (data/clean/test.en). Hãy kiểm tra lại.")
else:
    print("Chưa có checkpoint.")

## 11. Lưu kết quả (Output)
Kaggle sẽ tự động lưu lại các file trong `/kaggle/working` sau khi commit notebook.
Tuy nhiên bạn cũng có thể gộp file checkpoint lại để download cho dễ.

In [ ]:
# Nén thư mục checkpoints lại để tải về 1 lần
!zip -r checkpoints.zip checkpoints/

# 12. Generate Submission for Contest
Chạy cell bên dưới để sinh file dự đoán predictions.txt. Nhớ kiểm tra đường dẫn checkpoint.

In [ ]:
# === TẠO FILE SUBMISSION CHO CONTEST ===
# Chạy script để sinh file dự đoán từ model đã train
# Bạn có thể thay đổi đường dẫn checkpoint nếu muốn
!python scripts/generate_submission.py \
    --test_src data/clean/test.en \
    --output predictions.txt \
    --config configs/default.yaml \
    --checkpoint_path checkpoints/checkpoint_epoch_10.pt